# DD142X

### Prerequisites
* Python v3.7
* Tensorflow v1.4.2 
* Installera massa skit med pip för att få det att fungera
* Jag ändrade i train_image_classifier.py för att träna på CPU

Det kan funka med andra versioner, men inte Tensorflow 2.

### Ladda in directories

In [1]:
# Set pretrained checkpoint dir.
%env PRETRAINED_CHECKPOINT_DIR=/Users/johan/Dev/kexet/pretrained

# Where the training (fine-tuned) checkpoint and logs will be saved to.
%env TRAIN_DIR=/Users/johan/Dev/kexet/checkpoints

# Where the dataset is saved to.
%env DATASET_DIR=/Users/johan/Dev/kexet/data

env: PRETRAINED_CHECKPOINT_DIR=/Users/johan/Dev/kexet/pretrained
env: TRAIN_DIR=/Users/johan/Dev/kexet/checkpoints
env: DATASET_DIR=/Users/johan/Dev/kexet/data


In [6]:
### Download the model if you haven't yet.
!mkdir ${PRETRAINED_CHECKPOINT_DIR}
!wget http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz
!tar -xvf inception_v3_2016_08_28.tar.gz
!mv inception_v3.ckpt ${PRETRAINED_CHECKPOINT_DIR}
!rm inception_v3_2016_08_28.tar.gz

mkdir: /Users/johan/Dev/kexet/pretrained: File exists
--2020-05-08 15:44:32--  http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 216.58.211.16
Connecting to download.tensorflow.org (download.tensorflow.org)|216.58.211.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100885009 (96M) [application/x-tar]
Saving to: ‘inception_v3_2016_08_28.tar.gz’

inception_v3_2016_0 100%[===================>]  96,21M  17,0MB/s    in 5,8s    

2020-05-08 15:44:38 (16,5 MB/s) - ‘inception_v3_2016_08_28.tar.gz’ saved [100885009/100885009]

x inception_v3.ckpt


### Konvertera ISIC data

Kör en gång och gör sen manuellt skit.

In [1]:
import json
import os
import shutil

path_to_descriptions = '/Users/johan/Dev/kexet/rawdata'

final_path = '/Users/johan/Dev/kexet/data/skincancer'

for root, dirs, files in os.walk(path_to_descriptions):
    files = [f for f in files if not f[0] == '.']
    files = [f for f in files if not f[-1] == 'g']
    dirs[:] = [d for d in dirs if not d[0] == '.']
    
    for name in files:
        src = root + '/' + name
        try:
            with open(src) as data_file:
                #print(name)
                jsonList = json.load(data_file)

                disease = jsonList['meta']['clinical']['diagnosis']
                b_or_m = jsonList['meta']['clinical']['benign_malignant']
                if b_or_m is None:
                    b_or_m = 'neither'
                image_src_path = path_to_descriptions + '/Images/' + name + '.jpeg'
                disease_dir = final_path + '/' + disease + '/' + b_or_m
                if not os.path.exists(disease_dir):
                    os.makedirs(disease_dir)

                data_file.close()
                shutil.copy(image_src_path, disease_dir)
        except:
            continue
            

            
print("Finished!")

Finished!


In [ ]:
import download_and_convert_cancer as dcc
dcc.run("/Users/johan/Dev/kexet/data")

>> Converting image 1898/1898 shard 474
>> Converting image 4196/17080 shard 116

### Transfer learning
* Ladda vikterna från den generella färdigtränade modellen, men exkludera sista lagret med "checkpoint_exclude_scopes". 
* Träna om vikterna i detta lager med "trainable_scopes".

In [10]:
# Fine-tune only the new layers for 1000 steps.
!python3 train_image_classifier.py \
  --train_dir=${TRAIN_DIR} \
  --dataset_name=flowers \
  --dataset_split_name=train \
  --dataset_dir=${DATASET_DIR} \
  --model_name=inception_v3 \
  --checkpoint_path=${PRETRAINED_CHECKPOINT_DIR}/inception_v3.ckpt \
  --checkpoint_exclude_scopes=InceptionV3/Logits,InceptionV3/AuxLogits \
  --trainable_scopes=InceptionV3/Logits,InceptionV3/AuxLogits \
  --max_number_of_steps=1000 \
  --batch_size=32 \
  --learning_rate=0.01 \
  --learning_rate_decay_type=fixed \
  --save_interval_secs=600 \
  --save_summaries_secs=600 \
  --log_every_n_steps=100 \
  --optimizer=rmsprop \
  --weight_decay=0.00004



W0504 16:22:21.726774 4697810368 module_wrapper.py:139] From train_image_classifier.py:417: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0504 16:22:21.726970 4697810368 module_wrapper.py:139] From train_image_classifier.py:417: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

Instructions for updating:
Please switch to tf.train.create_global_step
W0504 16:22:21.727748 4697810368 deprecation.py:323] From train_image_classifier.py:431: create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.create_global_step

W0504 16:22:21.732081 4697810368 module_wrapper.py:139] From /Users/johan/Dev/kexet/models/research/slim/datasets/flowers.py:74: The name tf.FixedLenFeature is deprecated. Please use tf.io.FixedLenFeature instead.


W0504 16:22:21.732935 4697810368 module_

W0504 16:22:21.867448 4697810368 deprecation.py:323] From train_image_classifier.py:477: batch (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).

W0504 16:22:21.881979 4697810368 module_wrapper.py:139] From train_image_classifier.py:504: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.


W0504 16:22:21.882210 4697810368 module_wrapper.py:139] From train_image_classifier.py:504: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.


W0504 16:22:21.882524 4697810368 module_wrapper.py:139] From /Users/johan/Dev/kexet/models/research/slim/deployment/model_deploy.py:192: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.


W0504 16:22:21.882680 4697810368 module_wrappe


W0504 16:22:25.992835 4697810368 module_wrapper.py:139] From train_image_classifier.py:588: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.


W0504 16:22:26.004664 4697810368 module_wrapper.py:139] From train_image_classifier.py:363: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:Ignoring --checkpoint_path because a checkpoint already exists in /tmp/cancer-models/inception_v3
I0504 16:22:26.004827 4697810368 train_image_classifier.py:365] Ignoring --checkpoint_path because a checkpoint already exists in /tmp/cancer-models/inception_v3
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
W0504 16:22:26.529266 4697810368 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/contrib/slim/python/slim/learning.py:742: Supervisor.__init__ (from tensorflow.python.training.supervisor) is deprecated and will be removed in a future version.
Instructi

In [13]:
# Run evaluation.
!python3 eval_image_classifier.py \
  --checkpoint_path=${TRAIN_DIR} \
  --eval_dir=${TRAIN_DIR} \
  --dataset_name=flowers \
  --dataset_split_name=validation \
  --dataset_dir=${DATASET_DIR} \
  --model_name=inception_v3



W0504 16:58:31.171697 4503791040 module_wrapper.py:139] From eval_image_classifier.py:97: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0504 16:58:31.171890 4503791040 module_wrapper.py:139] From eval_image_classifier.py:97: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

Instructions for updating:
Please switch to tf.train.get_or_create_global_step
W0504 16:58:31.172437 4503791040 deprecation.py:323] From eval_image_classifier.py:99: get_or_create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step

W0504 16:58:31.176705 4503791040 module_wrapper.py:139] From /Users/johan/Dev/kexet/models/research/slim/datasets/flowers.py:74: The name tf.FixedLenFeature is deprecated. Please use tf.io.FixedLenFeature instead.


W0504 16:58:31.177387 450

Instructions for updating:
Please use `layer.__call__` method instead.
W0504 16:58:31.301645 4503791040 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/contrib/layers/python/layers/layers.py:1057: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Please switch to tf.metrics.accuracy. Note that the order of the labels and predictions arguments has been switched.
W0504 16:58:33.680132 4503791040 deprecation.py:323] From eval_image_classifier.py:167: streaming_accuracy (from tensorflow.contrib.metrics.python.ops.metric_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.metrics.accuracy. Note that the order of the labels and predictions arguments has been switched.
Instructions for updating:
Please use `streaming_sparse_recall_at_k`, and resh

INFO:tensorflow:Evaluation [1/4]
I0504 16:58:54.232089 4503791040 evaluation.py:167] Evaluation [1/4]
INFO:tensorflow:Evaluation [2/4]
I0504 16:59:07.400804 4503791040 evaluation.py:167] Evaluation [2/4]
INFO:tensorflow:Evaluation [3/4]
I0504 16:59:20.547014 4503791040 evaluation.py:167] Evaluation [3/4]
INFO:tensorflow:Evaluation [4/4]
I0504 16:59:33.619867 4503791040 evaluation.py:167] Evaluation [4/4]
eval/Accuracy[0.02]
eval/Recall_5[0.0325]
INFO:tensorflow:Finished evaluation at 2020-05-04-16:59:34
I0504 16:59:34.638002 4503791040 evaluation.py:275] Finished evaluation at 2020-05-04-16:59:34
